In [1]:
# Let's import our dependencies. 
import pandas as pd
import requests
import json
import numpy as np

# Import the API key
from config import geoapify_key  

In [2]:
# Read our CSV file into pandas dataframe. 

miami_df = pd.read_csv('miami-housing_2016.csv')
miami_df.head()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4


In [3]:
miami_df['LATITUDE_scale']= np.round(miami_df['LATITUDE'],3)
miami_df['LONGITUDE_scale']= np.round(miami_df['LONGITUDE'],3)

In [4]:
miami_df['LATITUDE_scale'] = miami_df['LATITUDE_scale'].map('{:.3f}'.format)
miami_df['LONGITUDE_scale'] = miami_df['LONGITUDE_scale'].map('{:.3f}'.format)

In [5]:
# Check the dataframe to ensure that there are no missing values. 
miami_df.head(10)
#miami_df.info()
#miami_df['LATITUDE_scale'].unique()
#miami_df['LONGITUDE_scale'].nunique()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality,LATITUDE_scale,LONGITUDE_scale
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4,25.891,-80.161
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4,25.891,-80.154
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4,25.891,-80.154
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4,25.892,-80.153
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4,25.892,-80.155
5,25.892060,-80.161354,622280070180,630000.0,9900,1531,2978,2391.4,13017.0,188.9,43135.1,38176.2,15687.2,41,0,2,4,25.892,-80.161
6,25.892473,-80.157217,622280080100,1020000.0,10387,1753,23116,3277.4,11667.8,0.0,43598.7,37973.9,17068.2,63,0,2,5,25.892,-80.157
7,25.893019,-80.157426,622280080400,850000.0,10272,1663,34933,3112.4,11718.1,10.5,43780.8,38198.3,16989.9,21,0,9,4,25.893,-80.157
8,25.893046,-80.161556,622280080020,250000.0,9375,1493,11668,2081.8,13043.8,51.5,43481.7,38542.0,15623.3,56,0,3,4,25.893,-80.162
9,25.893050,-80.158048,622280080370,1220000.0,13803,3077,34580,2937.7,11917.7,9.7,43730.1,38235.2,16787.0,63,0,11,5,25.893,-80.158


In [9]:
check_lat_lon =[]
lat_list = []
long_list =[]
school_count =[]
zip_list =[]
for index,row in miami_df.iterrows():
    lat_lon = row["LATITUDE_scale"]+row["LONGITUDE_scale"]
    if lat_lon not in check_lat_lon:
        check_lat_lon.append(lat_lon)
        categories = "education.school"
        radius = 1000
        filters = f'circle:{row["LONGITUDE_scale"]},{row["LATITUDE_scale"]},{radius}'
        bias = f'proximity:{row["LONGITUDE_scale"]},{row["LATITUDE_scale"]}'
        limit = 100

        # set up a parameters dictionary
        params = {
            "categories":categories,
            "bias":bias,
            "apiKey":geoapify_key,    
            "limit":limit,
            "filter":filters,
            "bias":bias,
        }
        base_url = "https://api.geoapify.com/v2/places"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)   
        # print the response url, avoid doing for public github repos in order to avoid exposing key
        print(response.url)   
        lat_list.append(row["LATITUDE_scale"])
        long_list.append(row["LONGITUDE_scale"])
        places_data = response.json()
        school_count.append(len(places_data["features"]))
        try:
            zip_list.append(places_data["features"][0]["properties"]["postcode"])
        except:
            print("No schools in this area")
            zip_list.append(0)

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.161%2C25.891&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.161%2C25.891%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.154%2C25.891&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.154%2C25.891%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.153%2C25.892&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.153%2C25.892%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.155%2C25.892&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.155%2C25.892%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.161%2C25.892&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.161%2C25.892%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.139%2C25.935&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.139%2C25.935%2C1000
No schools in this area
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.140%2C25.935&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.140%2C25.935%2C1000
No schools in this area
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.139%2C25.937&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.139%2C25.937%2C1000
No schools in this area
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.123%2C25.948&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.123%2C25.948%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.124%2C25.949&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.124%2C25.949%

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.179%2C25.895&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.179%2C25.895%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.176%2C25.896&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.176%2C25.896%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.172%2C25.896&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.172%2C25.896%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.173%2C25.896&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.173%2C25.896%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.175%2C25.896&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.175%2C25.896%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.174%2C25.907&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.174%2C25.907%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.183%2C25.907&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.183%2C25.907%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.186%2C25.907&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.186%2C25.907%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.184%2C25.907&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.184%2C25.907%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.181%2C25.908&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.181%2C25.908%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.170%2C25.915&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.170%2C25.915%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.176%2C25.915&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.176%2C25.915%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.173%2C25.916&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.173%2C25.916%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.169%2C25.916&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.169%2C25.916%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.170%2C25.916&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.170%2C25.916%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bi

...

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.198%2C25.812&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.198%2C25.812%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.196%2C25.812&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.196%2C25.812%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.201%2C25.813&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.201%2C25.813%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.203%2C25.813&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.203%2C25.813%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.196%2C25.813&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.196%2C25.813%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.221%2C25.813&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.221%2C25.813%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.213%2C25.813&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.213%2C25.813%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.221%2C25.814&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.221%2C25.814%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.214%2C25.814&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.214%2C25.814%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.221%2C25.815&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.221%2C25.815%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.203%2C25.807&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.203%2C25.807%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.200%2C25.807&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.200%2C25.807%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.201%2C25.807&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.201%2C25.807%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.198%2C25.807&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.198%2C25.807%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.197%2C25.807&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.197%2C25.807%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.125%2C25.878&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.125%2C25.878%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.129%2C25.879&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.129%2C25.879%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.124%2C25.879&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.124%2C25.879%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.127%2C25.879&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.127%2C25.879%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.125%2C25.879&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.125%2C25.879%2C1000
https://api.geoapify.com/v2/places?categories=education.school&bi

KeyboardInterrupt: 

In [11]:
len(school_count)

2127

In [30]:
schools_count_df = pd.DataFrame( {
    "LATITUDE_scale": lat_list,
    "LONGITUDE_scale": long_list,
    "count" : school_count,
    "postcodes" : zip_list
    })

In [31]:
schools_count_df.head(30)
print(schools_count_df['postcodes'].value_counts())

33162    228
33161    210
33147    188
33138    188
33168    169
33013    137
33142    132
33150    120
33054    119
33167     95
33012     94
33010     85
33127     82
33181     64
33137     34
0         31
33180     21
33141     20
33169     18
33151     17
33154     14
62600     10
33101      9
11890      9
33179      8
33136      5
33055      4
33166      4
33160      4
33125      3
33266      1
33128      1
33130      1
33132      1
33056      1
Name: postcodes, dtype: int64


In [32]:
schools_count_df.head()

,LATITUDE_scale,LONGITUDE_scale,count,postcodes
0,25.891,-80.161,3,33161
1,25.891,-80.154,1,33181
2,25.892,-80.153,1,33181
3,25.892,-80.155,1,33181
4,25.892,-80.161,3,33161


In [33]:
# Determine if there are any duplicate parcel numbers. 

miami_df["PARCELNO"].nunique()

# We see there are duplicate parcel numbers, but it seems that this is because the same property was sold twice (or more) in the same year.

13776

In [34]:
miami_df.columns

Index(['LATITUDE', 'LONGITUDE', 'PARCELNO', 'SALE_PRC', 'LND_SQFOOT',
       'TOT_LVG_AREA', 'SPEC_FEAT_VAL', 'RAIL_DIST', 'OCEAN_DIST',
       'WATER_DIST', 'CNTR_DIST', 'SUBCNTR_DI', 'HWY_DIST', 'age',
       'avno60plus', 'month_sold', 'structure_quality', 'LATITUDE_scale',
       'LONGITUDE_scale'],
      dtype='object')

In [38]:
school_merged_df = pd.merge(miami_df, schools_count_df, on = ['LATITUDE_scale','LONGITUDE_scale'])
school_merged_df
school_merged_df.sort_values(by=["count"],ascending=False)

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,...,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality,LATITUDE_scale,LONGITUDE_scale,count,postcodes
2074,25.836231,-80.224041,131140180750,84000.0,3600,1221,774,1650.2,32713.5,7831.6,...,23934.9,5941.3,36,0,8,1,25.836,-80.224,26,33147
2076,25.837078,-80.223592,131140181090,82000.0,3600,938,510,1345.0,32982.6,7488.0,...,24150.0,5785.5,66,0,10,2,25.837,-80.224,25,33147
2066,25.834653,-80.223537,131140270390,156500.0,6800,2355,861,2231.5,32117.6,8299.1,...,23341.0,5776.9,51,0,11,1,25.835,-80.224,24,33147
2071,25.835908,-80.217540,131140200140,169000.0,10800,2663,0,1829.6,32191.8,7260.7,...,22965.1,3774.8,66,0,12,1,25.836,-80.218,24,33147
2059,25.833806,-80.224044,131140270500,169000.0,3400,1433,496,2536.3,31859.1,8655.1,...,23147.1,5972.1,26,0,12,2,25.834,-80.224,23,33142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,25.867955,-80.174265,1132050190350,1450000.0,17550,3226,62833,2006.5,17947.7,1035.2,...,31508.0,11116.5,21,0,8,5,25.868,-80.174,0,0
1848,25.870461,-80.169775,1132050300140,1150000.0,9720,3109,20858,3047.3,16413.5,182.2,...,31706.1,12652.8,16,0,10,5,25.870,-80.170,0,0
1852,25.871259,-80.169720,1122320320020,930000.0,8925,2622,11863,2953.1,16364.9,0.0,...,31957.6,12682.4,36,0,1,5,25.871,-80.170,0,0
2092,25.813322,-80.262998,431200100221,240000.0,7100,1083,1740,857.9,30375.1,1013.8,...,23190.1,1671.8,26,1,5,4,25.813,-80.263,0,0


In [ ]:
hospital_count = []

for index,row in miami_df.iterrows():
    lat_lon = row["LATITUDE_scale"]+row["LONGITUDE_scale"]
    if lat_lon not in check_lat_lon:
        check_lat_lon.append(lat_lon)
        categories = "healthcare.hospital"
        radius = 1000
        filters = f'circle:{row["LONGITUDE_scale"]},{row["LATITUDE_scale"]},{radius}'
        bias = f'proximity:{row["LONGITUDE_scale"]},{row["LATITUDE_scale"]}'
        limit = 100

        # set up a parameters dictionary
        params = {
            "categories":categories,
            "bias":bias,
            "apiKey":geoapify_key,    
            "limit":limit,
            "filter":filters,
            "bias":bias,
        }
        base_url = "https://api.geoapify.com/v2/places"

        # run a request using our params dictionary
        hosp_response = requests.get(base_url, params=params)   
        # print the response url, avoid doing for public github repos in order to avoid exposing key
        print(hosp_response.url)   
        lat_list.append(row["LATITUDE_scale"])
        long_list.append(row["LONGITUDE_scale"])
        places_data = response.json()
        hjospital_count.append(len(places_data["features"]))
        try:
            zip_list.append(places_data["features"][0]["properties"]["postcode"])
        except:
            print("No hospitals in this area")
            zip_list.append(0)

In [22]:
# convert response to json
#places_data = response.json()

# Print the json (pretty printed)
#print(json.dumps(places_data, indent=4, sort_keys=True))

In [23]:
# convert response to json
#places_data = response.json()

# Print the json (pretty printed)
#print(json.dumps(places_data, indent=4, sort_keys=True))